<a href="https://colab.research.google.com/github/Nilufar-Komil/Machine-learning-EPFL-Fall-2024-/blob/main/ML_sheet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine Learning

# **Machine Learning**

This notebook contains my solutions to the tasks from ***Exercise Sheet~1*** of the course
*Machine Learning (CS-433), EPFL, Fall 2024*.

The original exercise sheet can be found [here](https://github.com/epfml/ML_course/blob/main/labs/ex01/exercise01.pdf).


**Task A: Matrix Standardization**

Write a function that accepts a data matrix
$x \in \mathbb{R}^{n \times d}$ as input and outputs the same data after normalization.
Here, $n$ is the number of samples and $d$ the number of dimensions,
i.e.\ rows contain samples and columns contain features.


In [ ]:
#importing needed libraries
import numpy as np

In [ ]:
# write down the function for task a
def normalize_matrix(x):
  x = x.copy()
  row, col = x.shape
  for j in range(col):
    x[:, j] = (x[:, j]-x[:, j].mean())/x[:, j].std()
  return x


In [ ]:
# testing

def test_normalize_matrix():
    # Example input
    X = np.array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]], dtype=float)

    # Call the function
    X_norm = normalize_matrix(X)

    # --- Tests ---
    # 1. Shape is preserved
    assert X_norm.shape == X.shape, "Shape changed after normalization!"

    # 2. Means are ~0 (allow tiny floating-point error)
    assert np.allclose(X_norm.mean(axis=0), 0, atol=1e-8), "Column means are not zero!"

    # 3. Standard deviations are ~1
    assert np.allclose(X_norm.std(axis=0), 1, atol=1e-8), "Column stds are not one!"

    print("✅ All tests passed!")

# Run the test
test_normalize_matrix()



✅ All tests passed!


**Task B: Pairwise Distances in the Plane**

Write a function that accepts two matrices
$P \in \mathbb{R}^{p \times 2}$ and $Q \in \mathbb{R}^{q \times 2}$
as input, where each row contains the $(x, y)$ coordinates of an interest point.
Note that the number of points ($p$ and $q$) do not have to be equal.
As output, compute the pairwise distances of all points in $P$ to all points in $Q$
and collect them in matrix $D$.
Element $D_{i,j}$ is the Euclidean distance of the $i$-th point in $P$
to the $j$-th point in $Q$.


In [ ]:
def euc_dist(P, Q):
    rowP, colP = P.shape
    rowQ, colQ = Q.shape
    dist_mat = np.zeros((rowP, rowQ))   # preallocate matrix

    for i in range(rowP):
        for j in range(rowQ):
            diff = P[i, :] - Q[j, :]
            dist_mat[i, j] = np.sqrt(np.sum(diff**2))

    return dist_mat

In [ ]:
# test function

def test_euc_dist():
    # Define small test cases where we know the result
    P = np.array([[0, 0],
                  [1, 1],
                  [2, 2]], dtype=float)

    Q = np.array([[0, 1],
                  [2, 0]], dtype=float)

    # Expected distances
    expected = np.array([
        [1.0, 2.0],                     # distances from (0,0)
        [1.0, np.sqrt(2)],              # distances from (1,1)
        [np.sqrt(5), 2.0]               # distances from (2,2)
    ])

    # Compute with the function
    D = euc_dist(P, Q)

    # --- Tests ---
    assert D.shape == expected.shape, "Shape of distance matrix is wrong!"
    assert np.allclose(D, expected, atol=1e-8), "Distance values are incorrect!"

    print("✅ All tests passed!")

# Run the test
test_euc_dist()


✅ All tests passed!


**Task C: Likelihood of a Data Sample**

The goal here is to practically implement the assignment of data to two given distributions in Python.
A subtask of many machine learning algorithms is to compute the likelihood $p(x_n \mid \theta)$ of a sample $x_n$
for a given density model with parameters $\theta$.
Given $k$ models, we now want to assign $x_n$ to the model for which the likelihood is maximal:

$$
a_n = \arg \max_m \, p(x_n \mid \theta_m), \quad m = 1, \ldots, k.
$$

Here $\theta_m = (\mu_m, \Sigma_m)$ are the parameters of the $m$-th density model
(with $\mu_m \in \mathbb{R}^d$ being the mean, and $\Sigma_m$ the covariance matrix).

We ask you to implement the assignment step for the two-model case, i.e.\ $k = 2$.
As input, your function receives a set of data examples $x_n \in \mathbb{R}^d$
(indexed by $1 \leq n \leq N$) as well as the two sets of parameters
$\theta_1 = (\mu_1, \Sigma_1)$ and $\theta_2 = (\mu_2, \Sigma_2)$
of two given multivariate Gaussian distributions:

$$
p(x_n \mid \mu, \Sigma) =
\frac{1}{(2\pi)^{d/2} \, |\Sigma|^{1/2}}
\exp\left( -\tfrac{1}{2} (x_n - \mu)^\top \Sigma^{-1} (x_n - \mu) \right),
$$

where $|\Sigma|$ is the determinant of $\Sigma$ and $\Sigma^{-1}$ its inverse.

Your function must return the ``most likely'' assignment $a_n \in \{1, 2\}$ for each input point $n$,
where $a_n = 1$ means that $x_n$ has been assigned to model~1.
In other words, in the case that $a_n = 1$, it holds that
$$
p(x_n \mid \mu_1, \Sigma_1) > p(x_n \mid \mu_2, \Sigma_2).
$$

